In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict, Counter
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF
from networkx.algorithms import bipartite

### FASE 1: COSTRUZIONE

* creare un grafo bipartito dalla edge list in ./data/crime/out.moreno_crime_crime
ATTENZIONE!! bisogna dividere i nodi people dai nodi crime perchè l'edge (1,1) non è un self loop ma indica che il primo nodo persona è legato al primo nodo crime, serve un mapping dei nodi!
* aggiungere ai nodi l'attributo 'name' dal file ./data/crime/ent.moreno_crime_crime.person.name
* aggiungere agli archi l'attributo 'crime-role' dal file ./data/crime/rel.moreno_crime_crime.person.role

* fare una proiezione sul piano delle persone: esiste un link tra due persone sse sono stati coinvolti in almeno un caso insieme. Mettere come peso il numero di crimini in comune. (potrebbe esistere una soluzione facile da nx)

In [2]:
bipartite_edge_list = open("../data/crime/out.moreno_crime_crime")
bipartite_edge_list = [
    (f"p{line.split()[0]}", f"c{line.split()[1]}") # Nodes are pX for people and cX for crime
    for line in list(bipartite_edge_list)[2:]
]

G = nx.Graph()

people = {u for u, v in bipartite_edge_list}
crime = {v for u, v in bipartite_edge_list}

# Add nodes with bipartite labels
G.add_nodes_from(people, bipartite='person')
G.add_nodes_from(crime, bipartite='crime')

G.add_edges_from(bipartite_edge_list)

bipartite.is_bipartite(G)

True

In [5]:
names = open("../data/crime/ent.moreno_crime_crime.person.name")
names = {
    f'p{idx}': name.strip()
    for idx, name in enumerate(names, start=1)
}

nx.set_node_attributes(G, names, 'name')

In [16]:
roles = open("../data/crime/rel.moreno_crime_crime.person.role")
roles = {
    bipartite_edge_list[idx] : role.strip()
    for idx, role in enumerate(roles, start=0)
}

nx.set_edge_attributes(G, roles, 'crime-role')

In [18]:
G_people = bipartite.weighted_projected_graph(G, people, ratio=False)

### FASE 2: ANALISI BIPARTITO

* Quante persone includono i crimini in media?
* In quanti crimini sono incluse le persone in media?
* In quanti crimini sono incluse le persone in media essendo witness?
* In media, quanti ruoli hanno le persone?
* Quanto spesso le persone hanno un ruolo solo? ( oppure, più difficile, il ruolo ricoperto più spesso dalle singole persone che percentula copre?)

In [31]:
crime_degrees = [G.degree(n) for n in crime]
avg_people_per_crime = np.mean(crime_degrees)

avg_people_per_crime

np.float64(2.678765880217786)

In [32]:
people_degrees = [G.degree(n) for n in people]
avg_crime_per_people = np.mean(people_degrees)

avg_crime_per_people

np.float64(1.7804583835946923)

In [41]:
witness_degrees = [
    len(list( # Count all edges that have an attribute Witness, for the node n
        edge
        for edge in G.edges(n, data=True)
        if edge[2]['crime-role'] == 'Witness'
    ))
    for n in people
]
avg_crime_per_witness = np.mean(witness_degrees)

avg_crime_per_witness

np.float64(0.23522316043425814)

In [45]:
roles_per_person = [
    len(set( # Count the different crime-roles from the people's edges
        edge[2]['crime-role']
        for edge in G.edges(n, data=True)
    ))
    for n in people
]

avg_roles_per_person = np.mean(roles_per_person)

avg_roles_per_person


np.float64(1.218335343787696)

In [50]:
one_role_people = len(list(
    x
    for x in roles_per_person
    if x == 1
))

one_role_people

682

In [65]:
roles_distribution = Counter(roles.values())
print(roles_distribution)

roles_distribution.most_common(1)[0][1] / roles_distribution.total()

Counter({'Suspect': 682, 'Victim': 558, 'Witness': 195, 'Victim Suspect': 41})


0.462059620596206

### FASE 3: ANALISI PROJECTED

* qual è la densità del grafo?
* il grafo è connesso?
* assegnando come ruole ad ogni nodo quello che assume più frequentemente, che ruole hanna maggiormente i 100 top nodi per degree centrality?

In [67]:
nx.density(G_people)

0.0065645705494659186

In [68]:
nx.is_connected(G_people)

False

In [86]:
degree_centrality = nx.degree_centrality(G_people)
top_100_by_degree_centrality = {
    x[0]
    for x in sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:100]
}

most_frequent_role =  {
    n: Counter( # Get the different crime-roles from the people's edges, then get the most common
        edge[2]['crime-role']
        for edge in G.edges(n, data=True)
    ).most_common(1)[0][0]
    for n in people
    if n in top_100_by_degree_centrality
}

roles_counter = Counter(most_frequent_role.values())
print(roles_counter)

roles_counter.most_common(1)[0]

Counter({'Suspect': 50, 'Victim': 41, 'Witness': 9})


('Suspect', 50)